<a href="https://colab.research.google.com/github/LaviWhite/ML/blob/main/Dissertation_Product_Detection(distilbert).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 83.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 69.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 53.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Dissertation/product_detection_1000.xlsx', index_col='index')

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size = 0.2, random_state= 2)

data_train, data_test = train_test_split(train, test_size = 0.25, random_state = 1)


In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train,
                                    text_column = 'content',
                                    label_columns = 'label',
                                    val_df = data_test,
                                    maxlen = 500,
                                    preprocess_mode = 'distilbert')

['not_label', 'label']
        not_label  label
index                   
1598          1.0    0.0
67625         1.0    0.0
20697         0.0    1.0
45581         0.0    1.0
107407        0.0    1.0
['not_label', 'label']
        not_label  label
index                   
123646        0.0    1.0
111103        0.0    1.0
114943        0.0    1.0
55933         0.0    1.0
43627         1.0    0.0
preprocessing train...
language: zh-cn


Is Multi-Label? False
preprocessing test...
language: zh-cn


ValueError: ignored

In [ ]:
## size of data
print(X_train[0].shape, y_train.shape)
print(X_test[0].shape, y_test.shape)

In [ ]:
model = text.text_classifier(name = 'distilbert', # or distilbert
                             train_data = (X_train, y_train),
                             preproc = preproc)

In [ ]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 16)

In [ ]:
learner.lr_find(show_plot=True, max_epochs=10)

In [ ]:
learner.fit_onecycle(lr = 0.0001, epochs = 4)
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/Disseration/distilbert-model')

In [ ]:
y_pred=predictor.predict(test['content'].values)
y_true = test['label'].values

In [ ]:
print(y_true)

In [ ]:
print(y_pred)

In [ ]:
for i in range(len(y_pred)):
    if y_pred[i] == 'not_label':
        y_pred[i] = 0
    else:
        y_pred[i] = 1

In [ ]:
print(y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))